In [2]:
%matplotlib inline

import pandas as pd
from matplotlib import pyplot as plt

# Enunciado

2)  Un  sitio  de  E
books  tiene  información  sobre  los  reviews  que  los  usuarios 
hacen  de  sus  libros  en  un  DataFrame  con  formato  (user_id,  book_id,  rating, 
timestamp). Por otro lado tenemos información en otro DataFrame que bajamos 
de  GoodReads:  (book_id,  book_name,  avg_rating)
.  Podemos  suponer  que  los 
Ids de los libros son compatibles. Se pide usar Python Pandas para:
-  Obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks. 
(Para este punto se puede ignorar el segundo DataFrame). (\*\*\*\*) (7.5 pts)
-  Obtener un DataFrame que indique qué libros tienen una diferencia de rating 
promedio mayor al 20% entre el sitio de Ebooks y GoodReads.  (\*\*\*) (7.5 pts)

# Creación de datos

In [7]:
d = {'user_id': [0, 0, 1, 2, 2, 2, 3, 3, 4, 4, 4], 'book_id': [0, 1, 6, 2, 3, 2, 7, 4, 0, 5, 5], 'rating': [1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1], 'timestamp': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
ebooks = pd.DataFrame(d)
ebooks

,book_id,rating,timestamp,user_id
0,0,1,0,0
1,1,2,1,0
2,6,3,2,1
3,2,4,3,2
4,3,5,4,2
5,2,1,5,2
6,7,2,6,3
7,4,3,7,3
8,0,4,8,4
9,5,5,9,4


In [8]:
d = {'book_id': [0, 1, 2, 3, 4, 5, 6, 7], 'book_name': [0, 1, 2, 3, 4, 5, 6, 7], 'avg_rating': [5, 2, 3, 5, 2, 4, 3, 2]}
goodreads = pd.DataFrame(d)
goodreads

,avg_rating,book_id,book_name
0,5,0,0
1,2,1,1
2,3,2,2
3,5,3,3
4,2,4,4
5,4,5,5
6,3,6,6
7,2,7,7


# TOP5 de Ebooks

In [9]:
ratings = ebooks.groupby('book_id').agg('mean').rating
ratings

book_id
0    2.5
1    2.0
2    2.5
3    5.0
4    3.0
5    3.0
6    3.0
7    2.0
Name: rating, dtype: float64

In [10]:
ratings = ratings.reset_index()
ratings

,book_id,rating
0,0,2.5
1,1,2.0
2,2,2.5
3,3,5.0
4,4,3.0
5,5,3.0
6,6,3.0
7,7,2.0


In [11]:
ratings.sort_values(by = 'rating', ascending=False).head()

,book_id,rating
3,3,5.0
4,4,3.0
5,5,3.0
6,6,3.0
0,0,2.5


In [12]:
ratings.nlargest(5, 'rating')

,book_id,rating
3,3,5.0
4,4,3.0
5,5,3.0
6,6,3.0
0,0,2.5


# Diferencia de rating mayor al 20%

In [18]:
ratings_ebooks = ebooks.groupby('book_id').agg('mean').rating
ratings_ebooks = pd.DataFrame(ratings_ebooks)
ratings_ebooks

,rating
book_id,
0,2.5
1,2.0
2,2.5
3,5.0
4,3.0
5,3.0
6,3.0
7,2.0


In [19]:
ratings_goodreads = goodreads.set_index('book_id').drop('book_name', axis = 1)
ratings_goodreads

,avg_rating
book_id,
0,5
1,2
2,3
3,5
4,2
5,4
6,3
7,2


In [20]:
ratings_join = ratings_ebooks.join(ratings_goodreads)
ratings_join

,rating,avg_rating
book_id,,
0,2.5,5
1,2.0,2
2,2.5,3
3,5.0,5
4,3.0,2
5,3.0,4
6,3.0,3
7,2.0,2


In [39]:
def diferencia(df):
    for i in range(0, len(df.index)):
        minimo = min(df['rating'][i],df['avg_rating'][i])
        maximo = max(df['rating'][i],df['avg_rating'][i])
        print(minimo*1.2 < maximo)
        if (minimo*1.2) < maximo:
            df['diferencia'][i] = True
        else:
            df['diferencia'][i] = False

diferencia(ratings_join)

True
False
False
False
True
True
False
False


/home/delfina/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/delfina/.local/lib/python3.5/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [46]:
ratings_join.loc[ratings_join['diferencia']].reset_index().book_id.values

array([0, 4, 5])